# Project - Market Day

<a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/"><img alt="Creative Commons License" align="left" src="https://i.creativecommons.org/l/by-sa/4.0/80x15.png" /></a>&nbsp;| Dennis G. Wilson | <a href="https://supaerodatascience.github.io/stochastic/">https://supaerodatascience.github.io/stochastic/</a>

![Stardew Valley](https://venturebeat.com/wp-content/uploads/2018/01/sw7rtba7p1xs77klsime.png)

It is Market Day! You've been working hard for a whole season, planting, watering, and taking care of crops. Now it is time to sell them in town. Just one problem though - your old truck can only carry so much weight. Which crops should you bring to maximize your profits? 

In this project, you will use a stochastic algorithm of your choice to find the best crop configuration you can load into your truck. You know how much stock you have of each crop, how much each crop weighs, and what price it will fetch at the market. You should use these values throughout the project.

Before we start, if you're using colab to run this notebook, you'll need to uncomment and run the following lines:

In [1]:
#!wget https://raw.githubusercontent.com/SupaeroDataScience/stochastic/master/project/market.csv
#!wget https://raw.githubusercontent.com/SupaeroDataScience/stochastic/master/project/recipes.csv
#!pip install pymoo

In [2]:
import numpy as np
import pandas as pd
import pymoo
import copy

In [3]:
df = pd.read_csv('market.csv', index_col=0)
df.head()

,stock,weight,price_A,price_B,price_C
Apple,5,5,1.00,0.5,1.1
Apricot,65,3,0.50,0.2,0.6
Blueberry,5,3,0.50,0.3,1.2
Cactus Fruit,90,4,0.75,0.3,0.3
Cherry,15,4,0.80,0.4,1.1


In [4]:
df.shape[0]


35

In [5]:
stock = df['stock'].to_dict()
weight = df['weight'].to_dict()
price = df['price_A'].to_dict()

In [6]:
stock

{'Apple': 5,
 'Apricot': 65,
 'Blueberry': 5,
 'Cactus Fruit': 90,
 'Cherry': 15,
 'Cranberry': 75,
 'Grape': 70,
 'Melon': 75,
 'Orange': 40,
 'Peach': 55,
 'Pomegranate': 5,
 'Rhubarb': 70,
 'Starfruit': 5,
 'Strawberry': 20,
 'Amaranth': 20,
 'Artichoke': 65,
 'Beet': 30,
 'Bok Choy': 55,
 'Cauliflower': 70,
 'Corn': 90,
 'Eggplant': 95,
 'Garlic': 95,
 'Green Bean': 55,
 'Hops': 75,
 'Hot Pepper': 25,
 'Kale': 5,
 'Parsnip': 10,
 'Potato': 55,
 'Pumpkin': 30,
 'Radish': 50,
 'Red Cabbage': 55,
 'Tomato': 65,
 'Wheat': 75,
 'Yam': 50,
 'Blackberry': 70}

To evaluate your possible load going to the market, we will use a function that takes in a dictionary of this type, specifying how many of each crop you will take. This function will take into the constraints of having enough stock of the crop type and of not surpassing the weight limit.

In [7]:
# Modifiable surtout si on veut retourner des valeurs négatives pour à quelle point la contrainte est brisée
def evaluate(load, stock, weight, price, max_weight=5000):
    total_weight = 0
    total_price = 0
    for k in load:
        if load[k] <= stock[k]:
            total_price += load[k] * price[k]
            total_weight += load[k] * weight[k]
            if total_weight > max_weight:
                return 0
        else:
            return 0
    return total_price

You can try this with an example load generated randomly:

In [8]:
trial_load = {}
for k in stock:
    trial_load[k] = np.random.randint(0, stock[k])
trial_load

{'Apple': 1,
 'Apricot': 22,
 'Blueberry': 1,
 'Cactus Fruit': 71,
 'Cherry': 11,
 'Cranberry': 58,
 'Grape': 22,
 'Melon': 7,
 'Orange': 8,
 'Peach': 46,
 'Pomegranate': 2,
 'Rhubarb': 15,
 'Starfruit': 3,
 'Strawberry': 10,
 'Amaranth': 17,
 'Artichoke': 5,
 'Beet': 16,
 'Bok Choy': 14,
 'Cauliflower': 47,
 'Corn': 48,
 'Eggplant': 81,
 'Garlic': 54,
 'Green Bean': 8,
 'Hops': 11,
 'Hot Pepper': 1,
 'Kale': 3,
 'Parsnip': 9,
 'Potato': 1,
 'Pumpkin': 17,
 'Radish': 13,
 'Red Cabbage': 40,
 'Tomato': 10,
 'Wheat': 73,
 'Yam': 34,
 'Blackberry': 50}

In [9]:
evaluate(trial_load, stock, weight, price)

694.65

## Challenge 1

Use a stochastic algorithm to find the configuration of crops that maximizes profit while respecting the constraints of the weight limit and stock. You can use any of the class code or a library like `pymoo`. You should create a reasonable representation for the problem and appropriate modification functions like mutation and crossover. You can also modify the evaluation function, as long as your final solution is valid according to the weight and stock constraints.

Include your code for optimization, any visualizations of the optimization process, as well as code to show the final market load and profit gained. Can you beat the random trial load profits?

Hint: this is a constrained case of the [knapsack problem](https://en.wikipedia.org/wiki/Knapsack_problem)

In [10]:
def evaluate(load, stock, weight, price, max_weight=5000):
    total_weight = 0
    total_price = 0
    for k in load:
        if load[k] <= stock[k]:
            total_price += load[k] * price[k]
            total_weight += load[k] * weight[k]
            if total_weight > max_weight:
                return 0
        else:
            return 0
    return total_price

In [11]:
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.operators.sampling.rnd import IntegerRandomSampling
from pymoo.operators.crossover.sbx import SBX
from pymoo.operators.mutation.pm import PM
from pymoo.operators.repair.rounding import RoundingRepair
from pymoo.operators.sampling.rnd import IntegerRandomSampling

algorithm = NSGA2(
    pop_size=100,
    n_offsprings=100,
    sampling=IntegerRandomSampling(),
    crossover=SBX(prob=1.0, eta=3.0, vtype=float, repair=RoundingRepair()),
    mutation=PM(prob=1.0, eta=3.0, vtype=float, repair=RoundingRepair()),
    eliminate_duplicates=True
)

In [45]:
from pymoo.core.problem import ElementwiseProblem

# Multi-Objective KnapSack Problem 
class MOKSP(ElementwiseProblem):
    
    def __init__(self, n_var, n_obj, max_weight=5000):      
        self.n_var = n_var
        self.max_weight = max_weight
        self.df = df[:self.n_var]
        
        print("Row count : ", df.shape[0])
        print(self.df)
        
        # lower bound
        xl = np.zeros(n_var)
        # upper bound
        xu = self.df['stock'].values
        
        self.price_keys = [x for x in df.columns.to_list() if "price" in x]
        assert 1 <= n_obj <= len(self.price_keys), "Too many n_obj"
        
        self.stock = self.df['stock'].values
        self.weight = self.df['weight'].values
        self.price = []
        
        for i in range(n_obj):
            self.price.append(self.df[self.price_keys[i]].values)
        
        super().__init__(n_var=n_var, n_obj=n_obj, n_constr=1, xl=xl, xu=xu)
    
    def compute_price_and_weight(self, load):
        total_weight = 0
        total_price = np.zeros(len(self.price))
        for i in range(len(self.price)):
            for k in range(load.shape[0]):
                total_price[i] += load[k] * self.price[i][k]
                total_weight += load[k] * self.weight[k]
        return total_weight, total_price
    
    def _evaluate(self, x, out, *args, **kwargs):

        total_weight, fits = self.compute_price_and_weight(x)
        
        fits = -fits # The optimizer only works with minimization
        
        if total_weight > self.max_weight:
            c = total_weight - self.max_weight
        else:
            c = 0
        
        # return by modifying out
        out["F"] = np.column_stack(fits)
        out["G"] = c

In [46]:
problem = MOKSP(35,1,5000)

Row count :  35
              stock  weight  price_A  price_B  price_C
Apple             5       5     1.00      0.5      1.1
Apricot          65       3     0.50      0.2      0.6
Blueberry         5       3     0.50      0.3      1.2
Cactus Fruit     90       4     0.75      0.3      0.3
Cherry           15       4     0.80      0.4      1.1
Cranberry        75       4     0.75      0.3      0.4
Grape            70       4     0.80      0.3      0.5
Melon            75      13     1.50      1.2      1.1
Orange           40       5     1.00      0.5      0.8
Peach            55       8     1.40      0.7      0.6
Pomegranate       5       8     1.40      0.8      1.2
Rhubarb          70      12     0.90      1.1      0.5
Starfruit         5      40     0.50      1.9      0.4
Strawberry       20       6     1.20      0.6      1.0
Amaranth         20       6     1.50      0.6      1.0
Artichoke        65       6     1.60      0.5      0.6
Beet             30       4     1.00      0.4    

In [47]:
from pymoo.optimize import minimize

res = minimize(problem,
               algorithm,
               ("n_gen", 1000),
               seed=1,
               save_history=True,
               verbose=True)

n_gen  |  n_eval  | n_nds  |     cv_min    |     cv_avg    |      eps      |   indicator  
     1 |      100 |      1 |  0.000000E+00 |  2.350000E+01 |             - |             -
     2 |      200 |      1 |  0.000000E+00 |  0.000000E+00 |  2.935000E+01 |         ideal
     3 |      300 |      1 |  0.000000E+00 |  0.000000E+00 |  1.760000E+01 |         ideal
     4 |      400 |      1 |  0.000000E+00 |  0.000000E+00 |  3.1000000000 |         ideal
     5 |      500 |      1 |  0.000000E+00 |  0.000000E+00 |  1.905000E+01 |         ideal
     6 |      600 |      1 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
     7 |      700 |      1 |  0.000000E+00 |  0.000000E+00 |  1.3000000000 |         ideal
     8 |      800 |      1 |  0.000000E+00 |  0.000000E+00 |  4.090000E+01 |         ideal
     9 |      900 |      1 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
    10 |     1000 |      1 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f

    93 |     9300 |      1 |  0.000000E+00 |  0.000000E+00 |  0.7000000000 |         ideal
    94 |     9400 |      1 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
    95 |     9500 |      1 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
    96 |     9600 |      1 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
    97 |     9700 |      2 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
    98 |     9800 |      1 |  0.000000E+00 |  0.000000E+00 |  1.2000000000 |         ideal
    99 |     9900 |      1 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
   100 |    10000 |      1 |  0.000000E+00 |  0.000000E+00 |  0.6500000000 |         ideal
   101 |    10100 |      1 |  0.000000E+00 |  0.000000E+00 |  0.3500000000 |         ideal
   102 |    10200 |      1 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
   103 |    10300 |      2 |  0.000000E+00 |  0.000000E+00 |  0.2000000000 |         ideal

   184 |    18400 |      1 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
   185 |    18500 |      1 |  0.000000E+00 |  0.000000E+00 |  0.0500000000 |         ideal
   186 |    18600 |      1 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
   187 |    18700 |      1 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
   188 |    18800 |      1 |  0.000000E+00 |  0.000000E+00 |  0.3500000000 |         ideal
   189 |    18900 |      1 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
   190 |    19000 |      1 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
   191 |    19100 |      1 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
   192 |    19200 |      1 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
   193 |    19300 |      1 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
   194 |    19400 |      1 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f

   278 |    27800 |      1 |  0.000000E+00 |  0.000000E+00 |  0.1500000000 |         ideal
   279 |    27900 |      1 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
   280 |    28000 |      1 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
   281 |    28100 |      1 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
   282 |    28200 |      1 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
   283 |    28300 |      1 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
   284 |    28400 |      1 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
   285 |    28500 |      1 |  0.000000E+00 |  0.000000E+00 |  0.1000000000 |         ideal
   286 |    28600 |      1 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
   287 |    28700 |      1 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
   288 |    28800 |      1 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f

   373 |    37300 |      1 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
   374 |    37400 |      3 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
   375 |    37500 |      1 |  0.000000E+00 |  0.000000E+00 |  0.0500000000 |         ideal
   376 |    37600 |      1 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
   377 |    37700 |      1 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
   378 |    37800 |      1 |  0.000000E+00 |  0.000000E+00 |  0.1500000000 |         ideal
   379 |    37900 |      1 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
   380 |    38000 |      1 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
   381 |    38100 |      1 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
   382 |    38200 |      1 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
   383 |    38300 |      1 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f

...

   561 |    56100 |      1 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
   562 |    56200 |      1 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
   563 |    56300 |      2 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
   564 |    56400 |      2 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
   565 |    56500 |      2 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
   566 |    56600 |      2 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
   567 |    56700 |      2 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
   568 |    56800 |      2 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
   569 |    56900 |      2 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
   570 |    57000 |      3 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
   571 |    57100 |      3 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f

   654 |    65400 |      3 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
   655 |    65500 |      3 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
   656 |    65600 |      3 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
   657 |    65700 |      3 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
   658 |    65800 |      3 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
   659 |    65900 |      3 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
   660 |    66000 |      3 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
   661 |    66100 |      3 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
   662 |    66200 |      3 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
   663 |    66300 |      3 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
   664 |    66400 |      3 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f

   745 |    74500 |      1 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
   746 |    74600 |      1 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
   747 |    74700 |      1 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
   748 |    74800 |      1 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
   749 |    74900 |      1 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
   750 |    75000 |      1 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
   751 |    75100 |      1 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
   752 |    75200 |      1 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
   753 |    75300 |      1 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
   754 |    75400 |      1 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
   755 |    75500 |      1 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f

   840 |    84000 |      1 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
   841 |    84100 |      1 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
   842 |    84200 |      1 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
   843 |    84300 |      1 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
   844 |    84400 |      1 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
   845 |    84500 |      1 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
   846 |    84600 |      1 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
   847 |    84700 |      1 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
   848 |    84800 |      1 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
   849 |    84900 |      1 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
   850 |    85000 |      1 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f

   931 |    93100 |      1 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
   932 |    93200 |      1 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
   933 |    93300 |      1 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
   934 |    93400 |      1 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
   935 |    93500 |      1 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
   936 |    93600 |      1 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
   937 |    93700 |      1 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
   938 |    93800 |      1 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
   939 |    93900 |      1 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
   940 |    94000 |      1 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
   941 |    94100 |      1 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f

In [54]:
print("Best score : ", -res.F[k])
print("Weight : ", problem.compute_price_and_weight(res.X[0])[0])
print("Matching load : \n", res.X[0])
print("In stock : \n", problem.stock)

# Best so far 1142

Best score :  [1140.15]
Weight :  5000
Matching load : 
 [ 5 23  0 90 15 75 70  0 40 45  2  0  0 20 20 65 30 55 70 90 95 95 55  0
 25  5 10 55  0 50 31 65  0 50 70]
In stock : 
 [ 5 65  5 90 15 75 70 75 40 55  5 70  5 20 20 65 30 55 70 90 95 95 55 75
 25  5 10 55 30 50 55 65 75 50 70]


In [ ]:
from pymoo.visualization.scatter import Scatter
plot = Scatter(tight_layout=True)
plot.add(res.F, s=10)
plot.show();

## Challenge 2


The agricultural market in this world is highly variable. This time, you'll be selling your load to a reseller, but you're not sure which one. Each reseller has different prices, so it depends on which one you'll meet. To be prepared, you should explore the possible options.

In [ ]:
price_a = df['price_A'].to_dict()
price_b = df['price_B'].to_dict()
price_c = df['price_C'].to_dict()

In [ ]:
price_a['Cherry'], price_b['Cherry'], price_c['Cherry']

Given that cherries have very different prices between the different resellers, the example load we made ealier would fetch a wildly different price depending on the reseller:

In [ ]:
a = evaluate(trial_load, stock, weight, price_a)
b = evaluate(trial_load, stock, weight, price_b)
c = evaluate(trial_load, stock, weight, price_c)
print(a, b, c)

Prepare four different options for your market day haul: one option that will be good for reseller A, one for reseller B, one for reseller C, and one which would be pretty good for all three. Display these options and compare their different prices.

Hint: you can use a multi-objective algorithm to optimize for all three reseller prices at the same time.

## Challenge 3

You decide to preprare some of your crops by making food for the market day. You know a number of recipes and are famous for your delicious fruit pies.

In [ ]:
recipes = {'Ratatouille': {'Eggplant': 2, 'Garlic': 2, 'Tomato': 4, 'Hot Pepper': 1},
 'Apple Pie': {'Apple': 10, 'Wheat': 5},
 'Apricot Pie': {'Apricot': 10, 'Wheat': 5},
 'Cherry Pie': {'Cherry': 10, 'Wheat': 5},
 'Rhubarb Pie': {'Rhubarb': 10, 'Wheat': 5},
 'Strawberry Pie': {'Strawberry': 10, 'Wheat': 5},
 'Blackberry Pie': {'Blackberry': 10, 'Wheat': 5},
 'Pumpkin Pie': {'Pumpkin': 10, 'Wheat': 5},
 'Pizza': {'Tomato': 3, 'Wheat': 2, 'Artichoke': 1},
 'Baba Ghanoush': {'Eggplant': 2, 'Garlic': 4},
 'Squash Soup': {'Yam': 3, 'Pumpkin': 1},
 'Peach Beer': {'Hops': 3, 'Peach': 1},
 'Blackberry Beer': {'Hops': 3, 'Blackberry': 1}}

These recipes sell for good prices at each of the resellers:

In [ ]:
df = pd.read_csv('recipes.csv', index_col=0)
df.head()

In [ ]:
price_a.update(df['price_A'].to_dict())
price_b.update(df['price_B'].to_dict())
price_c.update(df['price_C'].to_dict())
price_a['Apple Pie'], price_b['Apple Pie'], price_c['Apple Pie']

Modify the evaluation function to take into account these recipes, making sure not to break the weight or stock constraints. The weight of a recipe is the sum total weight of the ingredients, and you can not make a recipe if you don't have the remaining stock of ingredients. Here's an example of calculating the total weight of a random load of only recipes:

In [ ]:
recipe_load = {}
for k in recipes:
    recipe_load[k] = np.random.randint(0, 5)
recipe_load

In [ ]:
# Modif a faire avec le git
def get_weight(recipe_load, recipes, weight):
    total_weight = 0
    for k in recipe_load:
        ingredients = recipes[k]
        for i in ingredients:
            total_weight += weight[i] * ingredients[i]
    return total_weight

In [ ]:
get_weight(recipe_load, recipes, weight)

Once you've modified the evaluation function, rerun the optimization algorithm to find a new load for the three resellers, and one load which is good for all three. Display this result and the profit gained, making sure that the constraints are met.

## Evaluation

You should submit a saved copy of your notebook (including all of the cell output) to the LMS by December 13th, EOD. You may work with one partner, but you must **individually submit a notebook**. You will be graded based on your results, your code, and any text or visual explanations, according to the following rubric:

Criterion | Points
--- | ---
Results - Challenge 1 | 7
Results - Challenge 2 | 5
Results - Challenge 3 | 3
Presentation (code, text) | 5